In [1]:
# Standard-Bibliotheken
import warnings
import pandas as pd
import matplotlib.pyplot as plt

# Scikit-learn: Modellselektion und Validierung
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Scikit-learn: Feature-Engineering
from sklearn.feature_extraction.text import TfidfVectorizer

# Scikit-learn: Modelle
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn import tree 



# Scikit-learn: Metriken
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Scikit-learn: Pipeline
from sklearn.pipeline import Pipeline

# Warnungen ignorieren
warnings.simplefilter(action='ignore', category=FutureWarning)

# System
import os
import sys


### OPT 1: Laden Preprocess dateien (Daten Test u. Train seperat)
- inkl. Aufteilung X_train, X_test, y_train, y_test, X_dev, y_dev
- Bereitet die Trainings-, Test- und Validierungsdaten vor.
    Args:
        train_file (str): Pfad zur CSV-Datei der Trainingsdaten.
        test_file (str): Pfad zur CSV-Datei der Testdaten.
    Returns:
        tuple: Enthält die Trainingsdaten (X_train, y_train), 
               Testdaten (X_test, y_test) und 
               Validierungsdaten (X_dev, y_dev).

In [2]:
def prepare_data(train_file, test_file):
    # Daten laden
    data_train = pd.read_csv(train_file, on_bad_lines='skip', sep=';')
    data_test = pd.read_csv(test_file, on_bad_lines='skip', sep=';')

    # Features und Labels extrahieren
    X_train = data_train['text']
    y_train = data_train['group']
    X_test = data_test['text']
    y_test = data_test['group']

    # Testdaten in Test- und Validierungsdaten aufteilen
    X_test, X_dev, y_test, y_dev = train_test_split(
        X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
    )

    return (X_train, y_train), (X_test, y_test), (X_dev, y_dev)

OPT 1.1: Ausführung der prepare_data(): Laden dataLemmaLowerStop_train manuell
- Muss nicht ausgeführt werden wenn calculate_f1_macro() ausgeführt wird

In [13]:
train_file = "preprocessed/dataLemmaLowerStop_train.csv"
test_file = "preprocessed/dataLemmaLowerStop_test.csv"

(X_train, y_train), (X_test, y_test), (X_dev, y_dev) = prepare_data(train_file, test_file)

print("Trainingsdaten:", X_train.shape, y_train.shape)
print("Testdaten:", X_test.shape, y_test.shape)
print("Validierungsdaten:", X_dev.shape, y_dev.shape)

Trainingsdaten: (2624,) (2624,)
Testdaten: (328,) (328,)
Validierungsdaten: (328,) (328,)


OPT 1.2: Ausführung der prepare_data(): Laden dataLemmaLemmaStopNouns_train manuell
- Muss nicht ausgeführt werden wenn calculate_f1_macro() ausgeführt wird

In [ ]:
train_file = "preprocessed/dataLemmaLowerStopNouns_train.csv"
test_file = "preprocessed/dataLemmaLowerStopNouns_test.csv"

(X_train, y_train), (X_test, y_test), (X_dev, y_dev) = prepare_data(train_file, test_file)

print("Trainingsdaten:", X_train.shape, y_train.shape)
print("Testdaten:", X_test.shape, y_test.shape)
print("Validierungsdaten:", X_dev.shape, y_dev.shape)

### OPT 2 (Alte Variante): Laden Preprocess datei (Daten Test u. Train zsm.)
- inkl. Aufteilung X_train, X_test, y_train, y_test, X_dev, y_dev

In [ ]:
data = pd.read_csv("preprocessed/dataLemmaLowerStop.csv", on_bad_lines='skip', sep=';')
data = data.iloc[:,1:3]

X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['group'], test_size=0.2, random_state=42, stratify=data['group']
)

X_test, X_dev, y_test, y_dev = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_dev.shape)
print(y_dev.shape)

### OPT 3 (Alte Variante): Laden 20newgroups unprocessed
- inkl. Aufteilung X_train, X_test, y_train, y_test, X_dev, y_dev

In [ ]:
data = pd.read_csv("20 newsgroups/20newsgroups.csv", on_bad_lines='skip', sep=';')
data = data.dropna()
data = data.drop_duplicates()
data = data.iloc[:,1:3]

X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['group'], test_size=0.2, random_state=42, stratify=data['group']
)

X_test, X_dev, y_test, y_dev = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_dev.shape)
print(y_dev.shape)

### Tree Classifier ermittlung bester clf und tree Params (RandomizedSearchCV)
- Um herauszufinden welche Parameter mit GridSearchCV genauer untersucht werden müssen
- Zum testet n zufälliger Kombinationen der angegebenen tfidf und clf Parametern (n_iter), um eine Kombination mit gutem F1 Wert zu finden
- Verwendung Pipelin um TF-IDF-Vektorisierung und TreeClassifier zu vereinheitlichen

Erkentnisse:
- clf__criterion gini beim trainieren mit vektoren immer am besten
- min_sample_split 2 am besten
- min saplee_leaf 1 am besten

Erklärungen Parameter:
    Params TfidfVectorizer:
        - tfidf__max_df: Maximale Häufigkeit eines Begriffs in Dokumenten.
        - tfidf__min_df: Minimale Häufigkeit eines Begriffs in Dokumenten.
        - tfidf__max_features: Maximale Anzahl der zu extrahierenden Begriffe.
        - tfidf__sublinear_tf eine Möglichkeit, die Bedeutung häufiger Begriffe in den Dokumenten zu dämpfen
        - clf__max_depth: Maximale Tiefe des Entscheidungsbaums.
        - clf__min_samples_split: Minimale Anzahl an Proben, um einen Knoten zu teilen.
        - clf__criterion: Bewertungsfunktion für den Entscheidungsbaum. (entropy:feiner; gini:grober)
        - clf__min_samples_leaf: Minimale Anzahl an Proben in einem Blatt des Entscheidungsbaums.
    
    Params RandomizedSearchCV:
        - CV: Technik, um die Leistung eines Modells besser zu bewerten und Überanpassung (Overfitting) zu vermeiden. Dabei wird der verfügbare Datensatz in mehrere Teile (sogenannte Folds) aufgeteilt, und das Modell wird mehrfach trainiert und getestet.

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_df=0.8,
        min_df=2,
        ngram_range=(1, 2),
        stop_words='english',
        max_features=5000,
    )),
    ('clf', DecisionTreeClassifier(random_state=42))
])

param_distributions = {
    'tfidf__max_df': [0.1, 0.2, 0.5, 0.7, 0.8, 0.9],
    'tfidf__min_df': [1, 2, 5],
    'tfidf__max_features': [500, 1000, 3000, 5000, 10000, 15000, None],
    'tfidf__sublinear_tf': [True, False],
    'clf__max_depth': [60, 130, 140, 150, 160, 180, 200], 
    'clf__min_samples_split': [2, 3],
    'clf__min_samples_leaf': [1, 2],
    'clf__criterion': ['gini'],          
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=100,                   # Anzahl der zu testenden zufälligen Kombinationen
    cv=5,                         # 5-fache Cross-Validation
    scoring='f1_macro',           # Optimierung auf F1-Score (macro)
    random_state=42,              # Zufälligkeit kontrollieren
    n_jobs=-1                     # Parallele Verarbeitung
)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Beste Parameter:", best_params)

best_pipeline = random_search.best_estimator_

y_dev_pred = best_pipeline.predict(X_dev)

print("F1-Score: ", f1_score(y_dev, y_dev_pred, average='macro'))
print("Precision: ", precision_score(y_dev, y_dev_pred, average='macro'))
print("Recall: ", recall_score(y_dev, y_dev_pred, average='macro'))
print(classification_report(y_dev, y_dev_pred))

### Tree Classifier ermittlung bester clf und tree Params (GridSearchCV)

- Zum genauen testen aller Kombinationen der angegebenen tfidf und clf Parametern, um die mit dem besten F1 Wert zu finden
- Verwendung Pipelin um TF-IDF-Vektorisierung und TreeClassifier zu vereinheitlichen


Erklärungen Paramerter:
    Params TfidfVectorizer:
        - tfidf__max_df: Maximale Häufigkeit eines Begriffs in Dokumenten.
        - tfidf__min_df: Minimale Häufigkeit eines Begriffs in Dokumenten.
        - tfidf__max_features: Maximale Anzahl der zu extrahierenden Begriffe.
        - tfidf__sublinear_tf eine Möglichkeit, die Bedeutung häufiger Begriffe in den Dokumenten zu dämpfen
        - clf__max_depth: Maximale Tiefe des Entscheidungsbaums.
        - clf__min_samples_split: Minimale Anzahl an Proben, um einen Knoten zu teilen.
        - clf__criterion: Bewertungsfunktion für den Entscheidungsbaum. (entropy:feiner; gini:grober)
        - clf__min_samples_leaf: Minimale Anzahl an Proben in einem Blatt des Entscheidungsbaums.
    
    Params RandomizedSearchCV:
        - CV: Technik, um die Leistung eines Modells besser zu bewerten und Überanpassung (Overfitting) zu vermeiden. Dabei wird der verfügbare Datensatz in mehrere Teile (sogenannte Folds) aufgeteilt, und das Modell wird mehrfach trainiert und getestet.

In [3]:
def calculate_f1_macro_grid(train_file, test_file):

    (X_train, y_train), (X_test, y_test), (X_dev, y_dev) = prepare_data(train_file, test_file)

    pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_df=0.8,
        min_df=2,
        ngram_range=(1, 1), #TODO Anpassen für kontext => bisher (1,1) bestes Ergebnis
        stop_words='english',
        max_features=5000
    )),
    ('clf', DecisionTreeClassifier(random_state=42))
    ])

    # param_grid = {
    #     'tfidf__max_df': [0.05, 0.1, 0.15, 0.3],             
    #     'tfidf__min_df': [1, 2],                 
    #     'tfidf__max_features': [4500, 5000, 5600], 
    #     'tfidf__sublinear_tf': [True, False],           
    #     'clf__min_samples_split': [2, 3],           
    #     'clf__min_samples_leaf': [1, 2],            
    #     'clf__max_depth': [130, 190, 200],        
    #     'clf__criterion': ['gini'],          
    # }

    param_grid = {
        'tfidf__max_df': [0.05, 0.1, 0.3, 0.5],             
        'tfidf__min_df': [2],                 
        'tfidf__max_features': [500, 2500, 5600], 
        'tfidf__sublinear_tf': [True],           
        'clf__min_samples_split': [2],           
        'clf__min_samples_leaf': [1],            
        'clf__max_depth': [80, 130, 200],        
        'clf__criterion': ['gini'],          
    }

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,                          # 5-fache Cross-Validation
        scoring='f1_macro',            # Optimierung auf F1-Score (macro)
        n_jobs=-1                      # Parallele Verarbeitung
    )

    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_pipeline = grid_search.best_estimator_

    y_dev_pred = best_pipeline.predict(X_dev)
    
    f1_macro = f1_score(y_dev, y_dev_pred, average='macro')
    precision_macro = precision_score(y_dev, y_dev_pred, average='macro')
    recall_macro = recall_score(y_dev, y_dev_pred, average='macro')
    report = classification_report(y_dev, y_dev_pred)
    return f1_macro, precision_macro, recall_macro, report, best_pipeline, X_test, y_test, best_params

### calculate_f1_macro_grid() Methode zum berechnen des Tree Classifier mit Vektor (ohne Search)
- Zum testen der momentan besten Parameter 
- Parameter müssen direkt in Methode eintragen werden

In [49]:
train_file = "preprocessed/dataLemmaLowerStop_train.csv"
test_file = "preprocessed/dataLemmaLowerStop_test.csv"

f1_macro, precision_macro, recall_macro, report, pipeline, X_test, y_test, best_params = calculate_f1_macro_grid(train_file, test_file)

print("Beste Parameter:", best_params)
print("F1-Score: ", f1_macro)
print("Precision: ", precision_macro)
print("Recall: ", recall_macro)
print("Report: ", report)

Beste Parameter: {'clf__criterion': 'gini', 'clf__max_depth': 130, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'tfidf__max_df': 0.05, 'tfidf__max_features': 500, 'tfidf__min_df': 2, 'tfidf__sublinear_tf': True}
F1-Score:  0.6612402882070829
Precision:  0.6865728220419296
Recall:  0.6564946018893387
Report:                precision    recall  f1-score   support

           0       0.46      0.68      0.55        78
           1       0.86      0.81      0.83        95
           2       0.90      0.65      0.76        95
           3       0.53      0.48      0.50        60

    accuracy                           0.67       328
   macro avg       0.69      0.66      0.66       328
weighted avg       0.72      0.67      0.68       328



### calculate_f1_macro() Methode zum berechnen des Tree Classifier mit Vektor (ohne Search)
- Zum testen der momentan besten Parameter 
- Parameter müssen direkt in Methode eintragen werden

In [4]:
def calculate_f1_macro(train_file, test_file):

    (X_train, y_train), (X_test, y_test), (X_dev, y_dev) = prepare_data(train_file, test_file)

    # Nach Probe mit dataLemmaLowerStop.csv sind das die besten Parameter 
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            sublinear_tf=True,
            min_df=2,
            max_features=5600,
            ngram_range=(1, 1),
            max_df=0.05,
            stop_words='english'
        )),
        ('clf', DecisionTreeClassifier(
            random_state=42,
            min_samples_split=2,
            min_samples_leaf=1,
            max_depth=200,
            criterion='gini'
        ))
    ])

    pipeline.fit(X_train, y_train)
    y_dev_pred = pipeline.predict(X_dev)
    f1_macro = f1_score(y_dev, y_dev_pred, average='macro')
    precision_macro = precision_score(y_dev, y_dev_pred, average='macro')
    recall_macro = recall_score(y_dev, y_dev_pred, average='macro')
    report = classification_report(y_dev, y_dev_pred)
    return f1_macro, precision_macro, recall_macro, report, pipeline, X_test, y_test

Methode calculate_f1_macro Ausführen
- Ausgabe der f1_macro, precision_macro, recall_macro, pred werte für die in der Methode festgelegten Parameter

In [5]:
train_file = "preprocessed/dataLemmaLowerStop_train.csv"
test_file = "preprocessed/dataLemmaLowerStop_test.csv"

f1_macro, precision_macro, recall_macro, report, pipeline, X_test, y_test = calculate_f1_macro(train_file, test_file)

print("F1-Score: ", f1_macro)
print("Precision: ", precision_macro)
print("Recall: ", recall_macro)
print("Report: ", report)

F1-Score:  0.6960569199640945
Precision:  0.7128091328815769
Recall:  0.6923751686909582
Report:                precision    recall  f1-score   support

           0       0.69      0.53      0.60        78
           1       0.84      0.80      0.82        95
           2       0.62      0.81      0.70        95
           3       0.69      0.63      0.66        60

    accuracy                           0.71       328
   macro avg       0.71      0.69      0.70       328
weighted avg       0.72      0.71      0.70       328



### Training mit festen tree u. tfidf param für alle preprocesse Dateien
- Emittelt welche Preprocess Datei den besten F1 Score hat und gibt diesen aus

In [7]:
def train_on_directory(directory):
    train_files = [os.path.join(directory, f) for f in os.listdir(directory) if 'train' in f and f.endswith('.csv')]
    test_files = [os.path.join(directory, f) for f in os.listdir(directory) if 'test' in f and f.endswith('.csv')]
    results = pd.DataFrame(columns=["Train-Datei", "Test-Datei", "F1-Score", "Precision", "Recall"])
    
    best_f1 = 0
    best_file = None
    
    total_files = len(train_files)
    
    for i, train_file in enumerate(train_files, start=1):
        test_file = next((f for f in test_files if os.path.basename(train_file).replace('train', 'test') in f), None)
        if not test_file:
            print(f"Keine passende Testdatei für {train_file} gefunden, übersprungen.")
            continue
        
        progress = (i / total_files) * 100
        bar_length = 50
        filled_length = int(bar_length * i // total_files)
        bar = f"[{'#' * filled_length}{'.' * (bar_length - filled_length)}]"
        sys.stdout.write(f"\r{bar} {progress:.2f}% ({i}/{total_files})")
        sys.stdout.flush()
        
        ##f1, precision, recall, report, pipeline,_,_ = calculate_f1_macro(train_file, test_file)
        f1, precision, recall, report, pipeline,X_test, y_test,best_params = calculate_f1_macro_grid(train_file, test_file)


        results = pd.concat([results, pd.DataFrame({
            "Train-Datei": [train_file],
            "F1-Score": [f1],
            "Precision": [precision],
            "Recall": [recall],
            "Best Params": [best_params],
            "Pipeline" :[pipeline],
            "X_test" :[X_test],
            "y_test" :[y_test]
        })], ignore_index=True)
        
        if f1 > best_f1:
            best_f1 = f1
            best_file = train_file
            best_report = report
    
    results = results.sort_values(by="F1-Score", ascending=False)

    # Extrahieren der Top-5-Zeilen
    top_5_results = results.head(10)

    # Extrahieren der Pipeline-Objekte aus der Spalte "Pipeline"
    top_5_pipelines = top_5_results["Pipeline"].tolist()
    top_5_X_test = top_5_results["X_test"].values  
    top_5_y_test = top_5_results["y_test"].values


    print("\nErgebnisse:")
    top_5 = results.head(10)
    print("\nTop 5 Ergebnisse:")
    print(top_5)

    print(f"\nBester F1-Score: {best_f1}")
    print(f"Beste Datei: {best_file}")
    print(f"Report: {best_report}")
    return  top_5_X_test, top_5_y_test ,top_5_pipelines

directory = "preprocessed/"
top_5_X_test, top_5_y_test ,top_5_pipelines= train_on_directory(directory)

[###...............................................] 7.69% (1/13)

/home/dominik/vs-workspace/WiSe24/KI/projekt-git-abgabe/KI-Python/.venv/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[##################################################] 100.00% (13/13)
Ergebnisse:

Top 5 Ergebnisse:
                                  Train-Datei Test-Datei  F1-Score  Precision  \
2   preprocessed/dataLemmaLowerStop_train.csv        NaN  0.696057   0.712809   
12            preprocessed/dataStop_train.csv        NaN  0.667796   0.682991   
11           preprocessed/dataLower_train.csv        NaN  0.667796   0.682991   
8           preprocessed/dataNormal_train.csv        NaN  0.662322   0.684559   
7            preprocessed/dataKurze_train.csv        NaN  0.649586   0.661274   
10           preprocessed/dataSpace_train.csv        NaN  0.638856   0.654078   
1            preprocessed/dataLemma_train.csv        NaN  0.638856   0.654078   
3          preprocessed/dataUrlMail_train.csv        NaN  0.634925   0.655200   
5    preprocessed/dataSonderzeichen_train.csv        NaN  0.625765   0.636579   
4            preprocessed/dataKombi_train.csv        NaN  0.619481   0.629501   

      Re

### Ermittlung endgültiger F1-Score (an ungesehenen Testdaten)
- nach Ausführung train_on_directory

In [17]:
for i in range(4,9):
    y_dev_pred = top_5_pipelines[i].predict(top_5_X_test[i])
    print(top_5_pipelines[i])
    print("F1-Score: ", f1_score(top_5_y_test[i], y_dev_pred, average='macro'),)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.05, max_features=2500, min_df=2,
                                 stop_words='english', sublinear_tf=True)),
                ('clf',
                 DecisionTreeClassifier(max_depth=200, random_state=42))])
F1-Score:  0.6680730844475825
Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.05, max_features=2500, min_df=2,
                                 stop_words='english', sublinear_tf=True)),
                ('clf',
                 DecisionTreeClassifier(max_depth=200, random_state=42))])
F1-Score:  0.631583134206078
Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.05, max_features=2500, min_df=2,
                                 stop_words='english', sublinear_tf=True)),
                ('clf',
                 DecisionTreeClassifier(max_depth=200, random_state=42))])
F1-Score:  0.631583134206078
Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.05, max_features

### Ermittlung endgültiger F1-Score (an ungesehenen Testdaten)
- Bei Ausführung einzelner Berechnung

In [11]:
#TODO prüfen ob alle variablen korrekt verwewdet werden
y_dev_pred = pipeline.predict(X_test)
print("F1-Score: ", f1_score(y_test, y_dev_pred, average='macro'))
print("Precision: ", precision_score(y_test, y_dev_pred, average='macro'))
print("Recall: ", recall_score(y_test, y_dev_pred, average='macro'))
print(classification_report(y_test, y_dev_pred))

F1-Score:  0.7171292513855766
Precision:  0.7268508327830361
Recall:  0.7153014802281411
              precision    recall  f1-score   support

           0       0.68      0.58      0.63        77
           1       0.88      0.78      0.83        95
           2       0.68      0.84      0.75        95
           3       0.67      0.66      0.66        61

    accuracy                           0.73       328
   macro avg       0.73      0.72      0.72       328
weighted avg       0.74      0.73      0.73       328



### Importancewerte nach Training mit TreeClassifier ausgeben

In [1]:
clf = pipeline.named_steps['clf']  
tfidf = pipeline.named_steps['tfidf']  

feature_names = tfidf.get_feature_names_out()

importances = clf.feature_importances_

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("Wichtigste Features:")
print(importance_df.head(25))

importance_df.head(10).plot(kind="bar", x="Feature", y="Importance", legend=False)
plt.title("Top 10 wichtige Features im Entscheidungsbaum")
plt.show()

NameError: name 'pipeline' is not defined

### Importancewerte Top5 to csv

In [ ]:
# Iteriere durch die Top 5 Pipelines
for i in range(5,10):
    clf = top_5_pipelines[i].named_steps['clf']
    tfidf = top_5_pipelines[i].named_steps['tfidf']

    feature_names = tfidf.get_feature_names_out()

    importances = clf.feature_importances_


    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    importance_df.to_csv(f'tfidfWords/top_{i+1}_feature_importance.csv', index=False)


### Optional: Speichern TFIDF-Matrix in csv
- TODO: DataTrain public machen

In [ ]:
tfidf = pipeline.named_steps['tfidf']

tfidf_matrix = tfidf.transform(X_train)

tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(), 
    columns=tfidf.get_feature_names_out()
)

tfidf_df.to_csv("tfidf_matrix.csv", index=False, sep=';')
print("TF-IDF-Matrix wurde gespeichert.")

print(tfidf_df.head())


data_train['average_tfidf'] = tfidf_matrix.mean(axis=1)
average_per_class = data_train.groupby('group')['average_tfidf'].mean()
print("\nDurchschnittliche TF-IDF-Werte pro Klasse:\n", average_per_class)

### Optional: Histogramm Verteilung der TF-IDF-Werte plotten

In [ ]:
tfidf_values = tfidf_df.values.flatten()

plt.hist(tfidf_values, bins=50, color='blue', edgecolor='black', alpha=0.7)
plt.title("Häufigkeitsverteilung der TF-IDF-Werte")
plt.xlabel("TF-IDF-Wert")
plt.ylabel("Häufigkeit")
plt.yscale('log')  
plt.show()

### Optional: Entscheidungsbaum ploten

In [ ]:
clf = pipeline.named_steps['clf']  
tfidf = pipeline.named_steps['tfidf']  


plt.figure(figsize=(50,50))
tree.plot_tree(clf,feature_names=list(tfidf.get_feature_names_out()), fontsize=10, max_depth=3)